In [7]:
%cd ..

a:\Phong\Crawl_buffett


In [8]:
import PyPDF2
import re
import os
import time
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore", UserWarning)


In [9]:
from get_volume import convert_pdf_to_text


In [10]:
path = 'Data/5486/PDF/2008_Q1_決算短信(2008_7_30).pdf'

In [22]:
pd.read_csv("Data/9984/docs/dividend.csv")

,Year,Q1,Q2,Q3,Q4,time_split_Q1,time_split_Q2,time_split_Q3,time_split_Q4
0,2022,-,22.00,-,22.00,NaN,['9_12_2022'],NaN,['22_6_2023']
1,2021,-,22.00,-,22.00,NaN,['8_12_2021'],NaN,['27_6_2022']
2,2020,- 22.00 -,22.00,44.00,"79,592",[''],[''],[''],['24_6_2021']
3,2019,- 22.00 -,22.00,44.00,"91,063",[''],['9_12_2019'],[''],['26_6_2020']
4,2018,―,22.00,―,22.00,NaN,['B'],NaN,['20_6_2019']
5,2017,億ユーロ,22.5,11.7,11.9,NaN,['B'],['B'],['B']
6,2016,B,B,B,B,NaN,NaN,NaN,NaN
7,2015,B,B,B,B,NaN,NaN,NaN,NaN
8,2014,"74,402","76,614","2,212",3.0%,['B'],['B'],['B'],['B']
9,2013,B,B,B,B,NaN,NaN,NaN,NaN
